Using [`depfinder`](https://github.com/ericdill/depfinder) to discover all of the dependencies used on the [`deathbeds`](http://deathbeds.github.io) blog.  We will use `jupyter.services` and functional `pandas` programming to result the `dependencies` as structured data.

In [1]:
    import depfinder, pandas, pathlib, IPython, toolz.curried as toolz, notebook.services.contents.filemanager

In [2]:
    df = pandas.DataFrame(
        "From the deathbeds working directory" and
        notebook.services.contents.filemanager.FileContentsManager().get('')
    ).pipe(
        lambda df: "Expand the `df.content` from FileContentsManager." and 
        df.content.apply(pandas.Series).join(df, rsuffix='_')
    ).pipe(
        lambda df: "Consider only the posts with timestamps." and
        df[df.name.apply(toolz.compose(str.isnumeric, toolz.first))])

`type` the timestamps with `panda.to_datetime`.

In [3]:
    for key in ('last_modified', 'created'): df[key] = df[key].pipe(pandas.to_datetime)
    df = df.set_index(['created', 'last_modified', 'name'])

Apply `depfinder.notebook_path_to_dependencies` to each `df.path` and return different `dependency_types`.

In [4]:
    dependencies = df.path.apply(
        toolz.excepts(BaseException, depfinder.notebook_path_to_dependencies, lambda e: {})
    ).apply(pandas.Series)
    user_dependencies = ['required', 'questionable', 'relative']
    dependency_types = user_dependencies + ['builtin']
    df['builtin'] = dependencies['builtin']

We'll separate the dependency_types in `user_dependencies` and `df.builtin`.

In [5]:
    df['user_dependencies'] = pandas.concat([
        Series.apply(pandas.Series).stack().reset_index(-1, drop=1) for Series in 
        map(dependencies.__getitem__, user_dependencies)
    ]).sort_index().pipe(lambda  s: s.groupby(s.index).agg(set))
    

`df` has the following shape.

In [6]:
    df.sample(2)

,,,path,content,format,mimetype,size,writable,type,name_,path_,last_modified_,created_,content_,format_,mimetype_,size_,writable_,type_,builtin,user_dependencies
created,last_modified,name,,,,,,,,,,,,,,,,,,,
2018-07-03 13:25:03.756873+00:00,2018-08-18 13:52:19.230805+00:00,2018-06-23-Image-segmentation-on-flourishes.ipynb,2018-06-23-Image-segmentation-on-flourishes.ipynb,None,None,None,168715.0,True,notebook,,,2018-11-18 16:31:38.514833+00:00,2018-06-27 19:40:26.417050+00:00,{'name': '2018-06-23-Image-segmentation-on-flo...,json,None,None,True,directory,[io],"{importlib_resources, numpy, ipython, scikit-i..."
2018-08-05 14:00:34.746863+00:00,2018-08-14 19:58:46.873271+00:00,2018-08-04-Emojis-in-code-cells.ipynb,2018-08-04-Emojis-in-code-cells.ipynb,None,None,None,4312.0,True,notebook,,,2018-11-18 16:31:38.514833+00:00,2018-06-27 19:40:26.417050+00:00,{'name': '2018-08-04-Emojis-in-code-cells.ipyn...,json,None,None,True,directory,[ast],"{deathbeds, pytest}"


Create a tidy `timeline` of individual package usage.

In [7]:
    timeline = pandas.concat({
        key: df[key].dropna().apply(list).apply(lambda x: pandas.Series(dict(zip(x, [1]*len(x)))))
        for key in ['user_dependencies', 'builtin']
    }, axis=1).fillna(0).sort_index(0)
    timeline.sample(2)

builtin  \
                                                                                                              shelve   
created                          last_modified                    name                                                 
2018-09-07 14:54:58.532465+00:00 2018-09-07 15:07:57.834814+00:00 2018-09-07-Dataframes-and-modules.ipynb        0.0   
2018-09-19 13:52:14.749193+00:00 2018-09-19 13:52:14.775180+00:00 2018-09-18-Custom-Inspector-Behavior.ipynb     0.0   

                                                                                                                   \
                                                                                                             time   
created                          last_modified                    name                                              
2018-09-07 14:54:58.532465+00:00 2018-09-07 15:07:57.834814+00:00 2018-09-07-Dataframes-and-modules.ipynb     0.0   
2018-09-19 13:52:14.749193+00:00 2018-09-19 13:52:14.775180+00:00 2018-09-18-Custom-Inspector-Behavior.ipynb  0.0   

                                                                                                                   \
                                                                                                              abc   
created                          last_modified                    name                                              
2018-09-07 14:54:58.532465+00:00 2018-09-07 15:07:57.834814+00:00 2018-09-07-Dataframes-and-modules.ipynb     0.0   
2018-09-19 13:52:14.749193+00:00 2018-09-19 13:52:14.775180+00:00 2018-09-18-Custom-Inspector-Behavior.ipynb  0.0   

                                                                                                                   \
                                                                                                              ast   
created                          last_modified                    name                                              
2018-09-07 14:54:58.532465+00:00 2018-09-07 15:07:57.834814+00:00 2018-09-07-Dataframes-and-modules.ipynb     0.0   
2018-09-19 13:52:14.749193+00:00 2018-09-19 13:52:14.775180+00:00 2018-09-18-Custom-Inspector-Behavior.ipynb  0.0   

                                                                                                                      \
                                                                                                             doctest   
created                          last_modified                    name                                                 
2018-09-07 14:54:58.532465+00:00 2018-09-07 15:07:57.834814+00:00 2018-09-07-Dataframes-and-modules.ipynb        0.0   
2018-09-19 13:52:14.749193+00:00 2018-09-19 13:52:14.775180+00:00 2018-09-18-Custom-Inspector-Behavior.ipynb     0.0   

                                                                                                                   \
                                                                                                               os   
created                          last_modified                    name                                              
2018-09-07 14:54:58.532465+00:00 2018-09-07 15:07:57.834814+00:00 2018-09-07-Dataframes-and-modules.ipynb     0.0   
2018-09-19 13:52:14.749193+00:00 2018-09-19 13:52:14.775180+00:00 2018-09-18-Custom-Inspector-Behavior.ipynb  0.0   

                                                                                                                    \
                                                                                                             types   
created                          last_modified                    name                                               
2018-09-07 14:54:58.532465+00:00 2018-09-07 15:07:57.834814+00:00 2018-09-07-Dataframes-and-modules.ipynb      0.0   
2018-09-19 13:52:14.749193+00:00 2018-09-19 13:52:14.775180+00:00 2018-09-18-Custom-Inspect

The most frequently used builtins.

In [8]:
    timeline['builtin'].sum().sort_values(ascending=False).to_frame().T

,pathlib,ast,textwrap,functools,inspect,sys,abc,collections,io,json,...,argparse,time,unittest,tokenize,asyncio,linecache,itertools,warnings,builtins,shelve
0,19.0,16.0,10.0,9.0,9.0,8.0,7.0,7.0,7.0,7.0,...,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


The most frequently used user dependencies.

In [9]:
    timeline['user_dependencies'].sum().sort_values(ascending=False).to_frame().T

,ipython,deathbeds,pandas,pytest,toolz,ipywidgets,matplotlib,graphviz,importnb,disqus,...,stopit,delegator,hypothesis,Untitled,__style__,svgpathtools,_018_11_16_fspath,pythreejs,feather,lxml
0,61.0,36.0,31.0,17.0,16.0,14.0,10.0,9.0,9.0,9.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


# Summary

In [10]:
    IPython.display.Markdown(F"""## In total, {
        len(timeline.columns)
    } different dependencies were used ({
        len(timeline['builtin'].columns)
    } builtins and {
        len(timeline['user_dependencies'].columns)
    } user dependencies) {
        (pandas.to_datetime('now').tz_localize('EST') - df.index.get_level_values(0).min().tz_convert('EST')).days
    } over days of blogging and {
        len(df)
    } posts.""")

## In total, 98 different dependencies were used (32 builtins and 66 user dependencies) 144 over days of blogging and 109 posts.